#Unemployment Analysis - Introductory remarks here

In [69]:
# Importing dependencies

import pandas as pd
import numpy as np
import requests
import time

# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
#sns.set()

We chose to analyze unemployment data from the Federal Reserve Economic Data (FRED).
We accessed the information using an API key from Quandl.

In [ ]:
# Establishing the API call to retrieve raw data
# We are starting builing our dataframe using the "civilian" unemployment
# rate as a baseline or reference for the later analysis.

API_KEY = 'tT8qNAwBZW89z54wPsZy'
url = 'https://www.quandl.com/api/v3/datasets/FRED/UNRATE.json?api_key='+ API_KEY
response = requests.get(url).json()
response


In [ ]:
#Looking at results of the first API call and and creating a pandas dataframe

demographic = response['dataset']['name']
unemp_rate = response['dataset']['data']
unemp_rate
civilian = pd.DataFrame(unemp_rate,columns=['date' , demographic]) # converting list to df
civilian


In [80]:
# Each API call results in one series. We are interested in examining a set of 
# series representing demographic groups. Therefore, we created a function to
# make repeated API calls and the argument passed is a code for each demographic
# group. 

def my_func(a= 'LNS14000003'):
    
    API_KEY = 'tT8qNAwBZW89z54wPsZy'
    url = 'https://www.quandl.com/api/v3/datasets/FRED/'+a+'.json?api+key='+API_KEY
    response = requests.get(url).json()
    # print(response) to check results
    demographic = response['dataset']['name']
    unemp_rate = response['dataset']['data']
    df = pd.DataFrame(unemp_rate,columns=['date' , demographic])
    return df

In [ ]:
# Making the API calls for our demograpic groups of interest

# The first set of API calls is for race/ethnic groups

white = my_func(a = 'LNS14000003')
#white
hispanic = my_func(a = 'LNS14000009')
#hispanic
#print(response)
time.sleep(1.01)
asian = my_func(a= 'LNU04032183')
#asian
african = my_func(a='LNS14000006')
#african



In [90]:
# This group of API calls is for gender demographic groups

men = my_func(a= 'LNS14000001')
women= my_func(a= 'LNS14000002')

time.sleep(1.01)

KeyError: 'dataset'

In [ ]:
# This group of API calls is for age groups
# where age_1 is <25, age_2 is 25-34, age_3 is 35-44, age_4 is 45-54, age_5 is 55 and above.
# We use the time.sleep() method to delay the API calls when we are making several. 

age_1 = my_func(a= 'LNS14000036')
time.sleep(1.01)
age_2 = my_func(a= 'LNS14000089')
age_3 = my_func(a= 'LNS14000091')
age_4= my_func(a= 'LNS14000093')
age_5= my_func(a= 'LNS14024230')
time.sleep(1.01)

In [ ]:
# This group of API calls is for highest education level
# where edu_1 is did not complete high school, edu_2 is high school diploma, edu_3 is 
# bachelors degree, edu_4 is masters degree, edu_5 is PhD.

edu_1 = my_func(a= 'LNS14027659')
edu_2 = my_func(a= 'LNS14027660')
edu_3 = my_func(a= 'LNS14027662')
edu_4 = my_func(a= 'CGMD25O')
time.sleep(1.01)
edu_5 = my_func(a= 'CGDD25O')
time.sleep(1.01)


In [87]:
# Creating the larger master dataframe by peforming an outer merge ('m1')
# on the common column 'date'.

m1_df  = pd.merge(civilian,white,how='outer', on='date')
m1_df


,date,Civilian Unemployment Rate,Unemployment Rate: White
0,2021-03-01,6.0,5.4
1,2021-02-01,6.2,5.6
2,2021-01-01,6.3,5.7
3,2020-12-01,6.7,6.0
4,2020-11-01,6.7,5.9
...,...,...,...
874,1948-05-01,3.5,NaN
875,1948-04-01,3.9,NaN
876,1948-03-01,4.0,NaN
877,1948-02-01,3.8,NaN


In [88]:
m2_df=pd.merge(m1_df,hispanic, how='outer', on='date')
m3_df=pd.merge(m2_df,african,how='outer', on='date')
m3_df
m4_df=pd.merge(m3_df,asian,how='outer', on='date')
m4_df

,date,Civilian Unemployment Rate,Unemployment Rate: White,Unemployment Rate: Hispanic or Latino,Unemployment Rate: Black or African American,Unemployment Rate: Asian
0,2021-03-01,6.0,5.4,7.9,9.6,5.9
1,2021-02-01,6.2,5.6,8.5,9.9,5.1
2,2021-01-01,6.3,5.7,8.6,9.2,6.8
3,2020-12-01,6.7,6.0,9.3,9.9,5.8
4,2020-11-01,6.7,5.9,8.4,10.3,6.7
...,...,...,...,...,...,...
874,1948-05-01,3.5,NaN,NaN,NaN,NaN
875,1948-04-01,3.9,NaN,NaN,NaN,NaN
876,1948-03-01,4.0,NaN,NaN,NaN,NaN
877,1948-02-01,3.8,NaN,NaN,NaN,NaN


## Setting date as index

In [11]:
#un_race_df.set_index('date',inplace=True)

In [ ]:
#un_race_df.head()


In [ ]:
#unemploymentrate_race_df.head(50)

In [17]:
#unemploymentrate_race_df.shape

(879, 5)

### Unemployment Rate by Gender 

In [18]:
#def my_func(a= 'LNS14000001'):
    
#    API_KEY = 'tT8qNAwBZW89z54wPsZy'
#    url = 'https://www.quandl.com/api/v3/datasets/FRED/'+a+'.json?api+key='+API_KEY
#    response = requests.get(url).json()
#    response
#"https://www.quandl.com/api/v3/datasets/FRED/LNS14000003/
#    name = response['dataset']['name']
 #   data = response['dataset']['data']
#    df = pd.DataFrame(data,columns=['date' , name])
 #   return df


In [19]:
#unemploymentrate_men =my_func(a= 'LNS14000001')
#unemploymentrate_men 

#unemploymentrate_women= my_func(a= 'LNS14000002')
#unemploymentrate_women
#time.sleep(1.01)

In [89]:
gender_df= pd.merge(men,women,how='outer',on='date')
gender

NameError: name 'men' is not defined

In [ ]:
unemploymentrate_race_gender_df=pd.merge(unemploymentrate_race_df,unemploymentrate_gender,how='outer',on='date')
unemploymentrate_race_gender_df

## For converting date to datetime data type

In [ ]:
unemploymentrate_race_gender_df.dtypes

In [ ]:
unemploymentrate_race_gender_df["date"]=pd.to_datetime(unemploymentrate_race_gender_df["date"])
unemploymentrate_race_gender_df.dtypes

In [ ]:
unemploymentrate_race_gender_df.columns

## Renaming column names

In [ ]:
unemploymentrate_race_gender_df.rename(columns ={'date': 'Date','Unemployment Rate: Hispanic or Latino' :'Unemp_Hispanic',
                                                 'Unemployment Rate: Asian':'Unemp_Asian',
                                                 'Unemployment Rate: Black or African American':'Unemp_Black',
                                                'Unemployment Rate: White':'Unemp_White',
                                               'Civilian Unemployment Rate' :'Unemp_Civilian','Unemployment Rate: Men' :'Unemp_Men',
                                               'Unemployment Rate: Women':'Unemp_Women'},inplace=True)
unemploymentrate_race_gender_df

In [ ]:
unemploymentrate_race_gender_df

In [ ]:
unemploymentrate_race_gender_df["Unemp_White_Men"] =unemploymentrate_race_gender_df["Unemp_White"] * unemploymentrate_race_gender_df["Unemp_Men"]/1000*100
unemploymentrate_race_gender_df

In [ ]:
unemploymentrate_race_gender_df


## Unemployment Rate By Age

In [29]:
def my_func(a= 'LNS14000036'):
    
    API_KEY = 'tT8qNAwBZW89z54wPsZy'
    url = 'https://www.quandl.com/api/v3/datasets/FRED/'+a+'.json?api+key='+API_KEY
    response = requests.get(url).json()
    response
#"https://www.quandl.com/api/v3/datasets/FRED/LNS14000003/
    name = response['dataset']['name']
    data = response['dataset']['data']
    df = pd.DataFrame(data,columns=['date' , name])
    return df

In [ ]:
#df1 = my_func(a= 'LNS14000036')

#time.sleep(1.01)
#df1

In [ ]:
#df2 = my_func(a= 'LNS14000089')
#df2

In [ ]:
#df3 = my_func(a= 'LNS14000091')
#df3

In [ ]:
unemp_rate = pd.merge(df1, df2, how='left',on='date')
unemp_rate 

In [ ]:
unemp_rate1 = pd.merge(unemp_rate, df3, how='left',on='date')
unemp_rate1 

In [ ]:
#df4= my_func(a= 'LNS14000093')
#df4

In [ ]:
unemp_rate=pd.merge(unemp_rate1,df4,how='outer',on='date')
unemp_rate

In [37]:
#df5= my_func(a= 'LNS14024230')
#df5 
#time.sleep(1.01)

In [ ]:
unemp_rate_age = pd.merge(unemp_rate,df5,how='outer',on='date')
unemp_rate_age

In [ ]:
unemp_rate_age.columns

In [40]:
unemp_rate_age.rename(columns = {'date' :'Date',
                                 'Unemployment Rate: 20 to 24 years':'20-24' , 
                                 'Unemployment Rate: 25 to 34 years' :'25-34', 
                           'Unemployment Rate: 35 to 44 years': '35-44' ,
                               'Unemployment Rate: 45 to 54 years' : '45-54' , 
                                'Unemployment Rate: 55 years and over': '55 and up'},inplace=True)

In [ ]:
unemp_rate_age.columns

In [ ]:
unemp_rate_age

In [41]:
unemp_rate_age.dtypes

Date          object
20-24        float64
25-34        float64
35-44        float64
45-54        float64
55 and up    float64
dtype: object

In [42]:
unemp_rate_age["Date"] = pd.to_datetime(unemp_rate_age["Date"])


In [43]:
unemp_rate_age.dtypes

Date         datetime64[ns]
20-24               float64
25-34               float64
35-44               float64
45-54               float64
55 and up           float64
dtype: object

In [ ]:
unemprate_race_gender_age_df=pd.merge(unemploymentrate_race_gender_df,unemp_rate_age,how='outer',on='Date')
unemprate_race_gender_age_df

In [45]:
unemprate_race_gender_age_df.shape


(879, 14)

In [46]:
unemprate_race_gender_age_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 0 to 878
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             879 non-null    datetime64[ns]
 1   Unemp_Hispanic   577 non-null    float64       
 2   Unemp_Asian      255 non-null    float64       
 3   Unemp_Black      591 non-null    float64       
 4   Unemp_White      807 non-null    float64       
 5   Unemp_Civilian   879 non-null    float64       
 6   Unemp_Men        879 non-null    float64       
 7   Unemp_Women      879 non-null    float64       
 8   Unemp_White_Men  807 non-null    float64       
 9   20-24            879 non-null    float64       
 10  25-34            879 non-null    float64       
 11  35-44            879 non-null    float64       
 12  45-54            879 non-null    float64       
 13  55 and up        879 non-null    float64       
dtypes: datetime64[ns](1), float64(13)
memory u

## Unemployment Rate by Education

In [47]:
def my_func(a= 'LNS14000036'):
    
    API_KEY = 'tT8qNAwBZW89z54wPsZy'
    url = 'https://www.quandl.com/api/v3/datasets/FRED/'+a+'.json?api+key='+API_KEY
    response = requests.get(url).json()
    response
    time.sleep(1.01)
#"https://www.quandl.com/api/v3/datasets/FRED/LNS14000003/
    name = response['dataset']['name']
    data = response['dataset']['data']
    df = pd.DataFrame(data,columns=['date' , name])
    return df

In [48]:
#df1 = my_func(a= 'LNS14027659')
#df1

,date,"Unemployment Rate: Less than a High School Diploma, 25 years and over"
0,2021-03-01,8.2
1,2021-02-01,10.1
2,2021-01-01,9.1
3,2020-12-01,9.8
4,2020-11-01,9.2
...,...,...
346,1992-05-01,12.2
347,1992-04-01,10.8
348,1992-03-01,11.2
349,1992-02-01,11.0


In [49]:
#df2 = my_func(a= 'LNS14027660')
#df2

,date,"Unemployment Rate: High School Graduates, No College, 25 years and over"
0,2021-03-01,6.7
1,2021-02-01,7.2
2,2021-01-01,7.1
3,2020-12-01,7.8
4,2020-11-01,7.8
...,...,...
346,1992-05-01,6.9
347,1992-04-01,6.9
348,1992-03-01,6.9
349,1992-02-01,7.0


In [50]:
unemp_rate_edu = pd.merge(df1,df2,how='outer' ,on='date')
unemp_rate_edu


,date,"Unemployment Rate: Less than a High School Diploma, 25 years and over","Unemployment Rate: High School Graduates, No College, 25 years and over"
0,2021-03-01,8.2,6.7
1,2021-02-01,10.1,7.2
2,2021-01-01,9.1,7.1
3,2020-12-01,9.8,7.8
4,2020-11-01,9.2,7.8
...,...,...,...
346,1992-05-01,12.2,6.9
347,1992-04-01,10.8,6.9
348,1992-03-01,11.2,6.9
349,1992-02-01,11.0,7.0


In [51]:
#df3 = my_func(a= 'LNS14027662')
#df3

,date,"Unemployment Rate: College Graduates: Bachelor's Degree and Higher, 25 years and over"
0,2021-03-01,3.7
1,2021-02-01,3.8
2,2021-01-01,4.0
3,2020-12-01,3.8
4,2020-11-01,4.2
...,...,...
346,1992-05-01,3.2
347,1992-04-01,3.2
348,1992-03-01,2.9
349,1992-02-01,3.2


In [52]:
unemp_rate_edu1 = pd.merge(unemp_rate_edu,df3,on='date',how='outer')
unemp_rate_edu1

,date,"Unemployment Rate: Less than a High School Diploma, 25 years and over","Unemployment Rate: High School Graduates, No College, 25 years and over","Unemployment Rate: College Graduates: Bachelor's Degree and Higher, 25 years and over"
0,2021-03-01,8.2,6.7,3.7
1,2021-02-01,10.1,7.2,3.8
2,2021-01-01,9.1,7.1,4.0
3,2020-12-01,9.8,7.8,3.8
4,2020-11-01,9.2,7.8,4.2
...,...,...,...,...
346,1992-05-01,12.2,6.9,3.2
347,1992-04-01,10.8,6.9,3.2
348,1992-03-01,11.2,6.9,2.9
349,1992-02-01,11.0,7.0,3.2


In [53]:
#df4 = my_func(a= 'CGMD25O')
#time.sleep(1.01)
#df4

,date,"Unemployment Rate - College Graduates - Master's Degree, 25 years and over"
0,2021-03-01,3.6
1,2021-02-01,3.6
2,2021-01-01,3.7
3,2020-12-01,3.5
4,2020-11-01,3.5
...,...,...
250,2000-05-01,1.0
251,2000-04-01,1.2
252,2000-03-01,1.7
253,2000-02-01,1.6


In [54]:
unemp_rate_edu2=pd.merge(unemp_rate_edu1,df4,on='date',how='outer')
unemp_rate_edu2

,date,"Unemployment Rate: Less than a High School Diploma, 25 years and over","Unemployment Rate: High School Graduates, No College, 25 years and over","Unemployment Rate: College Graduates: Bachelor's Degree and Higher, 25 years and over","Unemployment Rate - College Graduates - Master's Degree, 25 years and over"
0,2021-03-01,8.2,6.7,3.7,3.6
1,2021-02-01,10.1,7.2,3.8,3.6
2,2021-01-01,9.1,7.1,4.0,3.7
3,2020-12-01,9.8,7.8,3.8,3.5
4,2020-11-01,9.2,7.8,4.2,3.5
...,...,...,...,...,...
346,1992-05-01,12.2,6.9,3.2,NaN
347,1992-04-01,10.8,6.9,3.2,NaN
348,1992-03-01,11.2,6.9,2.9,NaN
349,1992-02-01,11.0,7.0,3.2,NaN


In [55]:
#df5 = my_func(a= 'CGDD25O')
#time.sleep(1.01)
#df5

,date,"Unemployment Rate - College Graduates - Doctoral Degree, 25 years and over"
0,2021-03-01,1.6
1,2021-02-01,1.6
2,2021-01-01,2.3
3,2020-12-01,1.4
4,2020-11-01,1.9
...,...,...
250,2000-05-01,0.8
251,2000-04-01,0.6
252,2000-03-01,0.9
253,2000-02-01,1.4


In [56]:
unemp_rate_edu_df=pd.merge(unemp_rate_edu2,df5,on='date',how='outer')
unemp_rate_edu_df

,date,"Unemployment Rate: Less than a High School Diploma, 25 years and over","Unemployment Rate: High School Graduates, No College, 25 years and over","Unemployment Rate: College Graduates: Bachelor's Degree and Higher, 25 years and over","Unemployment Rate - College Graduates - Master's Degree, 25 years and over","Unemployment Rate - College Graduates - Doctoral Degree, 25 years and over"
0,2021-03-01,8.2,6.7,3.7,3.6,1.6
1,2021-02-01,10.1,7.2,3.8,3.6,1.6
2,2021-01-01,9.1,7.1,4.0,3.7,2.3
3,2020-12-01,9.8,7.8,3.8,3.5,1.4
4,2020-11-01,9.2,7.8,4.2,3.5,1.9
...,...,...,...,...,...,...
346,1992-05-01,12.2,6.9,3.2,NaN,NaN
347,1992-04-01,10.8,6.9,3.2,NaN,NaN
348,1992-03-01,11.2,6.9,2.9,NaN,NaN
349,1992-02-01,11.0,7.0,3.2,NaN,NaN


In [57]:
unemp_rate_edu_df.columns

Index(['date',
       'Unemployment Rate: Less than a High School Diploma, 25 years and over',
       'Unemployment Rate: High School Graduates, No College, 25 years and over',
       'Unemployment Rate: College Graduates: Bachelor's Degree and Higher, 25 years and over',
       'Unemployment Rate - College Graduates - Master's Degree, 25 years and over',
       'Unemployment Rate - College Graduates - Doctoral Degree, 25 years and over'],
      dtype='object')

In [58]:
unemp_rate_edu_df.columns = ['Date' ,'Unemp (>25, less than HS)', 'Unemp (>25, HS)' ,'Unemp (>25, College)' , 'Unemp (>25, Master)' ,'Unemp (>25, Doctoral)'] 

In [59]:
unemp_rate_edu_df

,Date,"Unemp (>25, less than HS)","Unemp (>25, HS)","Unemp (>25, College)","Unemp (>25, Master)","Unemp (>25, Doctoral)"
0,2021-03-01,8.2,6.7,3.7,3.6,1.6
1,2021-02-01,10.1,7.2,3.8,3.6,1.6
2,2021-01-01,9.1,7.1,4.0,3.7,2.3
3,2020-12-01,9.8,7.8,3.8,3.5,1.4
4,2020-11-01,9.2,7.8,4.2,3.5,1.9
...,...,...,...,...,...,...
346,1992-05-01,12.2,6.9,3.2,NaN,NaN
347,1992-04-01,10.8,6.9,3.2,NaN,NaN
348,1992-03-01,11.2,6.9,2.9,NaN,NaN
349,1992-02-01,11.0,7.0,3.2,NaN,NaN


In [60]:
unemp_rate_edu_df

,Date,"Unemp (>25, less than HS)","Unemp (>25, HS)","Unemp (>25, College)","Unemp (>25, Master)","Unemp (>25, Doctoral)"
0,2021-03-01,8.2,6.7,3.7,3.6,1.6
1,2021-02-01,10.1,7.2,3.8,3.6,1.6
2,2021-01-01,9.1,7.1,4.0,3.7,2.3
3,2020-12-01,9.8,7.8,3.8,3.5,1.4
4,2020-11-01,9.2,7.8,4.2,3.5,1.9
...,...,...,...,...,...,...
346,1992-05-01,12.2,6.9,3.2,NaN,NaN
347,1992-04-01,10.8,6.9,3.2,NaN,NaN
348,1992-03-01,11.2,6.9,2.9,NaN,NaN
349,1992-02-01,11.0,7.0,3.2,NaN,NaN


In [61]:
unemp_rate_edu_df.dtypes

Date                          object
Unemp (>25, less than HS)    float64
Unemp (>25, HS)              float64
Unemp (>25, College)         float64
Unemp (>25, Master)          float64
Unemp (>25, Doctoral)        float64
dtype: object

In [62]:
unemp_rate_edu_df['Date'] = pd.to_datetime(unemp_rate_edu_df['Date'])

In [63]:
unemp_rate_edu_df.dtypes

Date                         datetime64[ns]
Unemp (>25, less than HS)           float64
Unemp (>25, HS)                     float64
Unemp (>25, College)                float64
Unemp (>25, Master)                 float64
Unemp (>25, Doctoral)               float64
dtype: object

In [64]:
unemprate_race_gender_age_edu_df=pd.merge(unemprate_race_gender_age_df,unemp_rate_edu_df,on='Date',how='outer')
unemprate_race_gender_age_edu_df

,Date,Unemp_Hispanic,Unemp_Asian,Unemp_Black,Unemp_White,Unemp_Civilian,Unemp_Men,Unemp_Women,Unemp_White_Men,20-24,25-34,35-44,45-54,55 and up,"Unemp (>25, less than HS)","Unemp (>25, HS)","Unemp (>25, College)","Unemp (>25, Master)","Unemp (>25, Doctoral)"
0,2021-03-01,7.9,5.9,9.6,5.4,6.0,6.2,5.9,3.348,10.3,6.7,5.1,4.5,4.5,8.2,6.7,3.7,3.6,1.6
1,2021-02-01,8.5,5.1,9.9,5.6,6.2,6.3,6.1,3.528,9.6,6.4,5.7,4.9,5.3,10.1,7.2,3.8,3.6,1.6
2,2021-01-01,8.6,6.8,9.2,5.7,6.3,6.4,6.3,3.648,9.7,6.8,5.7,4.9,5.3,9.1,7.1,4.0,3.7,2.3
3,2020-12-01,9.3,5.8,9.9,6.0,6.7,6.7,6.7,4.020,11.2,6.6,5.5,5.3,6.0,9.8,7.8,3.8,3.5,1.4
4,2020-11-01,8.4,6.7,10.3,5.9,6.7,6.9,6.4,4.071,10.7,7.0,5.6,5.5,5.8,9.2,7.8,4.2,3.5,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,1948-05-01,NaN,NaN,NaN,NaN,3.5,3.5,3.7,NaN,6.6,3.0,2.6,2.4,3.1,NaN,NaN,NaN,NaN,NaN
875,1948-04-01,NaN,NaN,NaN,NaN,3.9,3.8,4.3,NaN,6.8,3.4,2.9,2.6,3.0,NaN,NaN,NaN,NaN,NaN
876,1948-03-01,NaN,NaN,NaN,NaN,4.0,3.8,4.4,NaN,7.6,2.9,2.7,2.6,3.0,NaN,NaN,NaN,NaN,NaN
877,1948-02-01,NaN,NaN,NaN,NaN,3.8,3.6,4.5,NaN,6.3,3.3,2.5,2.8,3.1,NaN,NaN,NaN,NaN,NaN


In [65]:
unemprate_race_gender_age_edu_df.shape

(879, 19)

## Unemployment Rate between a specified date range 

In [66]:
start_date = "2000-01-01"
### changed end_date from "2021-03-01"
end_date = "2010-03-01"

In [67]:
mask = (unemprate_race_gender_age_edu_df['Date'] > start_date) & (unemprate_race_gender_age_edu_df['Date'] <= end_date)
df = unemprate_race_gender_age_edu_df.loc[mask]
df

,Date,Unemp_Hispanic,Unemp_Asian,Unemp_Black,Unemp_White,Unemp_Civilian,Unemp_Men,Unemp_Women,Unemp_White_Men,20-24,25-34,35-44,45-54,55 and up,"Unemp (>25, less than HS)","Unemp (>25, HS)","Unemp (>25, College)","Unemp (>25, Master)","Unemp (>25, Doctoral)"
132,2010-03-01,12.9,7.5,16.8,8.9,9.9,10.9,8.7,9.701,15.8,10.2,8.6,7.9,6.9,14.9,11.0,4.9,3.9,1.4
133,2010-02-01,12.7,8.4,16.1,8.9,9.8,10.9,8.6,9.701,15.8,10.0,8.8,7.6,7.2,15.8,10.7,4.9,4.2,1.2
134,2010-01-01,12.9,8.4,16.5,8.8,9.8,11.0,8.4,9.680,15.9,10.0,8.5,7.8,6.9,15.3,10.2,4.9,4.4,1.9
135,2009-12-01,12.8,8.4,16.1,9.0,9.9,10.9,8.7,9.810,15.5,10.0,8.7,7.8,7.2,15.0,10.6,4.9,3.9,2.7
136,2009-11-01,12.4,7.3,15.7,9.2,9.9,11.0,8.6,10.120,16.0,10.3,8.6,7.6,7.0,14.7,10.3,4.8,4.2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2000-06-01,5.6,4.9,7.8,3.4,4.0,3.8,4.1,1.292,7.5,3.6,3.1,2.5,2.3,6.3,3.4,1.6,1.6,0.7
250,2000-05-01,5.8,3.5,7.7,3.5,4.0,3.9,4.2,1.365,8.0,3.8,3.0,2.4,2.5,6.9,3.4,1.6,1.0,0.8
251,2000-04-01,5.5,3.1,7.0,3.4,3.8,3.7,4.0,1.258,7.2,3.5,3.0,2.1,2.3,6.2,3.3,1.5,1.2,0.6
252,2000-03-01,6.1,4.1,7.4,3.5,4.0,3.8,4.3,1.330,7.4,3.7,3.2,2.2,2.7,6.6,3.3,1.7,1.7,0.9


## Impact of covid on Unemployment Rate 

In [78]:
start_date = "2020-03-01"
end_date = "2021-03-01"

In [79]:
mask = (unemprate_race_gender_age_edu_df['Date'] > start_date) & (unemprate_race_gender_age_edu_df['Date'] <= end_date)
df = unemprate_race_gender_age_edu_df.loc[mask]
df

,Date,Unemp_Hispanic,Unemp_Asian,Unemp_Black,Unemp_White,Unemp_Civilian,Unemp_Men,Unemp_Women,Unemp_White_Men,20-24,25-34,35-44,45-54,55 and up,"Unemp (>25, less than HS)","Unemp (>25, HS)","Unemp (>25, College)","Unemp (>25, Master)","Unemp (>25, Doctoral)"
0,2021-03-01,7.9,5.9,9.6,5.4,6.0,6.2,5.9,3.348,10.3,6.7,5.1,4.5,4.5,8.2,6.7,3.7,3.6,1.6
1,2021-02-01,8.5,5.1,9.9,5.6,6.2,6.3,6.1,3.528,9.6,6.4,5.7,4.9,5.3,10.1,7.2,3.8,3.6,1.6
2,2021-01-01,8.6,6.8,9.2,5.7,6.3,6.4,6.3,3.648,9.7,6.8,5.7,4.9,5.3,9.1,7.1,4.0,3.7,2.3
3,2020-12-01,9.3,5.8,9.9,6.0,6.7,6.7,6.7,4.020,11.2,6.6,5.5,5.3,6.0,9.8,7.8,3.8,3.5,1.4
4,2020-11-01,8.4,6.7,10.3,5.9,6.7,6.9,6.4,4.071,10.7,7.0,5.6,5.5,5.8,9.2,7.8,4.2,3.5,1.9
5,2020-10-01,8.8,7.5,10.8,6.0,6.9,7.0,6.8,4.200,10.9,7.3,5.8,6.0,5.5,9.9,8.1,4.2,3.6,1.9
6,2020-09-01,10.3,8.8,12.0,7.0,7.8,7.7,8.0,5.390,12.5,8.6,6.2,6.4,6.6,10.7,9.0,4.7,3.9,2.0
7,2020-08-01,10.5,10.7,12.8,7.4,8.4,8.3,8.6,6.142,14.1,9.6,6.4,6.2,7.6,12.6,9.8,5.3,4.9,3.7
8,2020-07-01,12.7,12.2,14.4,9.2,10.2,9.8,10.6,9.016,18.1,11.3,8.0,7.8,8.7,15.1,10.8,6.7,6.0,3.6
9,2020-06-01,14.5,13.9,15.3,10.1,11.1,10.5,11.7,10.605,19.6,11.7,9.0,8.4,9.6,16.4,12.2,6.9,5.6,4.5


In [83]:
unemprate_race_gender_age_edu_df

,Date,Unemp_Hispanic,Unemp_Asian,Unemp_Black,Unemp_White,Unemp_Civilian,Unemp_Men,Unemp_Women,Unemp_White_Men,20-24,25-34,35-44,45-54,55 and up,"Unemp (>25, less than HS)","Unemp (>25, HS)","Unemp (>25, College)","Unemp (>25, Master)","Unemp (>25, Doctoral)",unemp_gender
0,2021-03-01,7.9,5.9,9.6,5.4,6.0,6.2,5.9,3.348,10.3,6.7,5.1,4.5,4.5,8.2,6.7,3.7,3.6,1.6,12.1
1,2021-02-01,8.5,5.1,9.9,5.6,6.2,6.3,6.1,3.528,9.6,6.4,5.7,4.9,5.3,10.1,7.2,3.8,3.6,1.6,12.4
2,2021-01-01,8.6,6.8,9.2,5.7,6.3,6.4,6.3,3.648,9.7,6.8,5.7,4.9,5.3,9.1,7.1,4.0,3.7,2.3,12.7
3,2020-12-01,9.3,5.8,9.9,6.0,6.7,6.7,6.7,4.020,11.2,6.6,5.5,5.3,6.0,9.8,7.8,3.8,3.5,1.4,13.4
4,2020-11-01,8.4,6.7,10.3,5.9,6.7,6.9,6.4,4.071,10.7,7.0,5.6,5.5,5.8,9.2,7.8,4.2,3.5,1.9,13.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,1948-05-01,NaN,NaN,NaN,NaN,3.5,3.5,3.7,NaN,6.6,3.0,2.6,2.4,3.1,NaN,NaN,NaN,NaN,NaN,7.2
875,1948-04-01,NaN,NaN,NaN,NaN,3.9,3.8,4.3,NaN,6.8,3.4,2.9,2.6,3.0,NaN,NaN,NaN,NaN,NaN,8.1
876,1948-03-01,NaN,NaN,NaN,NaN,4.0,3.8,4.4,NaN,7.6,2.9,2.7,2.6,3.0,NaN,NaN,NaN,NaN,NaN,8.2
877,1948-02-01,NaN,NaN,NaN,NaN,3.8,3.6,4.5,NaN,6.3,3.3,2.5,2.8,3.1,NaN,NaN,NaN,NaN,NaN,8.1


In [88]:
unemprate_race_gender_age_edu_df.drop(columns=['unemp_gender' ],inplace=True)

Which race has the  highest unemployment rate between covid ?


In [89]:
unemprate_race_gender_age_edu_df

,Date,Unemp_Hispanic,Unemp_Asian,Unemp_Black,Unemp_White,Unemp_Civilian,Unemp_Men,Unemp_Women,Unemp_White_Men,20-24,25-34,35-44,45-54,55 and up,"Unemp (>25, less than HS)","Unemp (>25, HS)","Unemp (>25, College)","Unemp (>25, Master)","Unemp (>25, Doctoral)"
0,2021-03-01,7.9,5.9,9.6,5.4,6.0,6.2,5.9,3.348,10.3,6.7,5.1,4.5,4.5,8.2,6.7,3.7,3.6,1.6
1,2021-02-01,8.5,5.1,9.9,5.6,6.2,6.3,6.1,3.528,9.6,6.4,5.7,4.9,5.3,10.1,7.2,3.8,3.6,1.6
2,2021-01-01,8.6,6.8,9.2,5.7,6.3,6.4,6.3,3.648,9.7,6.8,5.7,4.9,5.3,9.1,7.1,4.0,3.7,2.3
3,2020-12-01,9.3,5.8,9.9,6.0,6.7,6.7,6.7,4.020,11.2,6.6,5.5,5.3,6.0,9.8,7.8,3.8,3.5,1.4
4,2020-11-01,8.4,6.7,10.3,5.9,6.7,6.9,6.4,4.071,10.7,7.0,5.6,5.5,5.8,9.2,7.8,4.2,3.5,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,1948-05-01,NaN,NaN,NaN,NaN,3.5,3.5,3.7,NaN,6.6,3.0,2.6,2.4,3.1,NaN,NaN,NaN,NaN,NaN
875,1948-04-01,NaN,NaN,NaN,NaN,3.9,3.8,4.3,NaN,6.8,3.4,2.9,2.6,3.0,NaN,NaN,NaN,NaN,NaN
876,1948-03-01,NaN,NaN,NaN,NaN,4.0,3.8,4.4,NaN,7.6,2.9,2.7,2.6,3.0,NaN,NaN,NaN,NaN,NaN
877,1948-02-01,NaN,NaN,NaN,NaN,3.8,3.6,4.5,NaN,6.3,3.3,2.5,2.8,3.1,NaN,NaN,NaN,NaN,NaN


In [98]:
# Save data frame to CSV
#weather_data.to_csv('Output_CSV/weather_data.csv')
unemprate_race_gender_age_edu_df.to_csv (r'C:\Users\sunny\Documents\project-repository\unemp_rate.csv', index = False, header=True)
unemprate_race_gender_age_edu_df

,Date,Unemp_Hispanic,Unemp_Asian,Unemp_Black,Unemp_White,Unemp_Civilian,Unemp_Men,Unemp_Women,Unemp_White_Men,20-24,25-34,35-44,45-54,55 and up,"Unemp (>25, less than HS)","Unemp (>25, HS)","Unemp (>25, College)","Unemp (>25, Master)","Unemp (>25, Doctoral)"
0,2021-03-01,7.9,5.9,9.6,5.4,6.0,6.2,5.9,3.348,10.3,6.7,5.1,4.5,4.5,8.2,6.7,3.7,3.6,1.6
1,2021-02-01,8.5,5.1,9.9,5.6,6.2,6.3,6.1,3.528,9.6,6.4,5.7,4.9,5.3,10.1,7.2,3.8,3.6,1.6
2,2021-01-01,8.6,6.8,9.2,5.7,6.3,6.4,6.3,3.648,9.7,6.8,5.7,4.9,5.3,9.1,7.1,4.0,3.7,2.3
3,2020-12-01,9.3,5.8,9.9,6.0,6.7,6.7,6.7,4.020,11.2,6.6,5.5,5.3,6.0,9.8,7.8,3.8,3.5,1.4
4,2020-11-01,8.4,6.7,10.3,5.9,6.7,6.9,6.4,4.071,10.7,7.0,5.6,5.5,5.8,9.2,7.8,4.2,3.5,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,1948-05-01,NaN,NaN,NaN,NaN,3.5,3.5,3.7,NaN,6.6,3.0,2.6,2.4,3.1,NaN,NaN,NaN,NaN,NaN
875,1948-04-01,NaN,NaN,NaN,NaN,3.9,3.8,4.3,NaN,6.8,3.4,2.9,2.6,3.0,NaN,NaN,NaN,NaN,NaN
876,1948-03-01,NaN,NaN,NaN,NaN,4.0,3.8,4.4,NaN,7.6,2.9,2.7,2.6,3.0,NaN,NaN,NaN,NaN,NaN
877,1948-02-01,NaN,NaN,NaN,NaN,3.8,3.6,4.5,NaN,6.3,3.3,2.5,2.8,3.1,NaN,NaN,NaN,NaN,NaN


## Reading the data from csv file

In [105]:
unemp_data=pd.read_csv("unemp_rate.csv")
unemp_data
unemp_data.set_index('Date',inplace=True)
unemp_data

,Unemp_Hispanic,Unemp_Asian,Unemp_Black,Unemp_White,Unemp_Civilian,Unemp_Men,Unemp_Women,Unemp_White_Men,20-24,25-34,35-44,45-54,55 and up,"Unemp (>25, less than HS)","Unemp (>25, HS)","Unemp (>25, College)","Unemp (>25, Master)","Unemp (>25, Doctoral)"
Date,,,,,,,,,,,,,,,,,,
2021-03-01,7.9,5.9,9.6,5.4,6.0,6.2,5.9,3.348,10.3,6.7,5.1,4.5,4.5,8.2,6.7,3.7,3.6,1.6
2021-02-01,8.5,5.1,9.9,5.6,6.2,6.3,6.1,3.528,9.6,6.4,5.7,4.9,5.3,10.1,7.2,3.8,3.6,1.6
2021-01-01,8.6,6.8,9.2,5.7,6.3,6.4,6.3,3.648,9.7,6.8,5.7,4.9,5.3,9.1,7.1,4.0,3.7,2.3
2020-12-01,9.3,5.8,9.9,6.0,6.7,6.7,6.7,4.020,11.2,6.6,5.5,5.3,6.0,9.8,7.8,3.8,3.5,1.4
2020-11-01,8.4,6.7,10.3,5.9,6.7,6.9,6.4,4.071,10.7,7.0,5.6,5.5,5.8,9.2,7.8,4.2,3.5,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948-05-01,NaN,NaN,NaN,NaN,3.5,3.5,3.7,NaN,6.6,3.0,2.6,2.4,3.1,NaN,NaN,NaN,NaN,NaN
1948-04-01,NaN,NaN,NaN,NaN,3.9,3.8,4.3,NaN,6.8,3.4,2.9,2.6,3.0,NaN,NaN,NaN,NaN,NaN
1948-03-01,NaN,NaN,NaN,NaN,4.0,3.8,4.4,NaN,7.6,2.9,2.7,2.6,3.0,NaN,NaN,NaN,NaN,NaN


In [110]:
cols = [col for col in unemp_data.columns]
fig, ax = plt.subplots(1, 1, figsize=(14,6))
plt.style.use('fivethirtyeight')

for c in cols:
    ax.plot(c, linewidth=2, data=df)
    ax.set_title('Unemployment Rate by Race')
    plt.xlabel('Year')
    plt.ylabel('Percent')
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels,loc='upper left', prop={"size":10})    
plt.show()

NameError: name 'plt' is not defined